In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv("coronanet_release_Italy.csv")

In [4]:
df


,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,...,travel_mechanism,compliance,enforcer,dist_index_high_est,dist_index_med_est,dist_index_low_est,dist_index_country_rank,link,date_updated,recorded_date
0,R_1CI4xlvjnlHWQmYNA,9064618,new_entry,original,NaN,NaN,"February 4, 2020: The Minister of Health Rober...",2020-02-04,2020-02-04,2021-05-04,...,NaN,Voluntary/Recommended but No Penalties,"Ministry/Department of Health,Other (Please sp...",24.623557,18.462258,12.944540,122.0,http://www.salute.gov.it/portale/nuovocoronavi...,2021-04-08,2021-04-08T19:14:19Z
1,R_3NO3XfVzgoeNnXENA,3115620,new_entry,original,NaN,NaN,"February 7, 2020: The Minister of Health Rober...",2020-02-07,2020-02-07,2021-05-04,...,NaN,Voluntary/Recommended but No Penalties,"National Government,Other (Please specify in t...",26.981839,20.215196,15.011816,128.0,http://www.salute.gov.it/portale/nuovocoronavi...,2021-04-08,2021-04-08T19:22:29Z
2,R_33d9f2OBQ4W5uvQNA,4990959,new_entry,original,NaN,NaN,"February 28, 2020: The Minister of Health Robe...",2020-02-28,2020-02-28,2021-05-04,...,NaN,Voluntary/Recommended but No Penalties,"Ministry/Department of Health,Other (Please sp...",68.381885,64.969683,61.708645,137.0,http://www.salute.gov.it/portale/nuovocoronavi...,2021-04-08,2021-04-08T19:32:41Z
3,R_2X5tIPEhYrQnseGNA,7018250,new_entry,original,NaN,NaN,"March 28, 2020: The Government has instituted ...",2020-03-28,2020-03-28,2021-05-04,...,NaN,Voluntary/Recommended but No Penalties,"Ministry/Department of Health,New (COVID-19 sp...",85.389356,82.532188,79.556456,135.0,https://www.repubblica.it/politica/2020/04/05/...,2021-04-08,2021-04-08T19:44:37Z
4,R_3G96WnKTKZwxquiNA,8198592,new_entry,correction,NaN,NaN,"April 14, 2020: The Italian Ministry of Health...",2020-04-14,2020-04-14,NaN,...,NaN,Voluntary/Recommended but No Penalties,Ministry/Department of Health,83.695753,80.610608,77.438278,128.0,http://www.salute.gov.it/portale/nuovocoronavi...,2021-04-28,2021-04-28T15:28:46Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,R_3PX0J3sXqRLnwihEz,5516145,new_entry,original,NaN,NaN,"March 4, 2021: With Ordinanza N. 18, the Presi...",2021-03-04,2021-03-06,2021-03-22,...,NaN,Mandatory with Exceptions (Please list excepti...,Provincial/State Government,NaN,NaN,NaN,NaN,https://pti.regione.sicilia.it/portal/page/por...,2021-04-16,2021-04-16T13:21:45Z
2209,R_3PX0J3sXqRLnwihFa,5516145,new_entry,original,NaN,NaN,"March 4, 2021: With Ordinanza N. 18, the Presi...",2021-03-04,2021-03-06,2021-03-22,...,NaN,Mandatory with Exceptions (Please list excepti...,Provincial/State Government,NaN,NaN,NaN,NaN,https://pti.regione.sicilia.it/portal/page/por...,2021-04-16,2021-04-16T13:21:45Z
2210,R_3PX0J3sXqRLnwihEy,5516145,new_entry,original,NaN,NaN,"March 4, 2021: With Ordinanza N. 18, the Presi...",2021-03-04,2021-03-06,2021-03-22,...,NaN,Mandatory with Exceptions (Please list excepti...,Provincial/State Government,NaN,NaN,NaN,NaN,https://pti.regione.sicilia.it/portal/page/por...,2021-04-16,2021-04-16T13:21:45Z
2211,R_3PX0J3sXqRLnwihFh,5516145,new_entry,original,NaN,NaN,"March 4, 2021: With Ordinanza N. 18, the Presi...",2021-03-04,2021-03-06,2021-03-22,...,NaN,Mandatory with Exceptions (Please list excepti...,Provincial/State Government,NaN,NaN,NaN,NaN,https://pti.regione.sicilia.it/portal/page/por...,2021-04-16,2021-04-16T13:21:45Z


In [24]:
sorted(df.columns)

['ISO_A2',
 'ISO_A3',
 'ISO_L2',
 'city',
 'compliance',
 'correct_type',
 'country',
 'date_announced',
 'date_end',
 'date_start',
 'date_updated',
 'description',
 'dist_index_country_rank',
 'dist_index_high_est',
 'dist_index_low_est',
 'dist_index_med_est',
 'domestic_policy',
 'enforcer',
 'entry_type',
 'init_country_level',
 'institution_status',
 'link',
 'policy_id',
 'province',
 'record_id',
 'recorded_date',
 'target_city',
 'target_country',
 'target_direction',
 'target_geog_level',
 'target_other',
 'target_province',
 'target_region',
 'target_who_what',
 'travel_mechanism',
 'type',
 'type_sub_cat',
 'type_text',
 'update_level',
 'update_type']

In [31]:
df[ df["ISO_L2"].isnull()]["province"].unique()

array([nan, 'Liguria', 'Latium', 'The Marches', 'Lombardy',
       'Friuli Venezia Giulia', 'Pennsylvania'], dtype=object)

In [6]:
df[df["entry_type"] == "update"]["update_type"].unique()

array([nan, 'Change of Policy', 'End of Policy'], dtype=object)

In [14]:
df[df.date_end.isnull() & (df.entry_type == "update")  ]["type"].unique()

array(['Closure and Regulation of Schools',
       'External Border Restrictions', 'Health Resources',
       'Internal Border Restrictions', 'Lockdown',
       'Other Policy Not Listed Above', 'Quarantine',
       'Restriction and Regulation of Businesses', 'Social Distancing'],
      dtype=object)

In [17]:
def aggr_policy(group):
          group = group.sort_values(by=['date_start', 'date_announced', 'date_end'])
          init = group[group['entry_type']=='new_entry']
          for _, row in group.iterrows():
            if row['entry_type'] == 'update':
              if row['update_type'] == 'End of Policy':
                if row['date_end'] is not None:
                  init['date_end'] = row['date_end']
                else:
                  if row['date_start']>row['date_announced']:
                    init['date_end'] = row['date_start']
                  else:
                    init['date_end'] = row['date_announced']
                break
              else:
                if row['date_end'] is not None:
                  init['date_end'] = row['date_end']
          return init

df = df.groupby('policy_id').apply(aggr_policy)

In [21]:
df[df.date_end.isnull()  ]["type"].unique()

array(['Restriction and Regulation of Government Services',
       'Social Distancing', 'Restriction and Regulation of Businesses',
       'Health Resources',
       'New Task Force, Bureau or Administrative Configuration',
       'Quarantine', 'Closure and Regulation of Schools',
       'Health Testing', 'Public Awareness Measures', 'Hygiene',
       'Other Policy Not Listed Above', 'Health Monitoring',
       'Internal Border Restrictions', 'External Border Restrictions',
       'Restrictions of Mass Gatherings', 'Lockdown',
       'Declaration of Emergency', 'Anti-Disinformation Measures'],
      dtype=object)